<a href="https://colab.research.google.com/github/lokeshvirani/Internship-Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1  **Import Libraries**




In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances


In [16]:
# Load and preprocess MNIST dataset
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

     # Normalize the images range [0, 1] and expand dimensions
    x_train = np.expand_dims(x_train, axis=-1) / 255.0
    x_test = np.expand_dims(x_test, axis=-1) / 255.0

    # Resize images to match ResNet50 input size
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))

    # Convert to RGB by repeating the grey scale channel 3 time
    x_train = np.repeat(x_train, 3, axis=-1)
    x_test = np.repeat(x_test, 3, axis=-1)

    return (x_train, y_train), (x_test, y_test)

# Initialize ResNet50 as  feature extractor
def initialize_feature_extractor():
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    resnet_model.trainable = False  # Freeze model layers
    return resnet_model

# Feeature extraction using ResNet50
def extract_features(model, data):
    return model.predict(data, verbose=1)

# Prototype Management:
class PrototypeManager:
    def __init__(self):
        self.prototypes = {}
        self.sample_counts = {}

    def update_or_create_prototype(self, class_label, feature_vector):
        if class_label in self.prototypes:
            current_prototype = self.prototypes[class_label]
            current_count = self.sample_counts[class_label]
            new_prototype = (current_prototype * current_count + feature_vector) / (current_count + 1)
            self.prototypes[class_label] = new_prototype
            self.sample_counts[class_label] += 1
        else:
            self.prototypes[class_label] = feature_vector
            self.sample_counts[class_label] = 1

    def find_closest_prototype(self, feature_vector, threshold=0.5):
        min_distance = float('inf')
        closest_class = None
    # Distance Measurement
        for class_label, prototype in self.prototypes.items():
            distance = np.linalg.norm(prototype - feature_vector)
            if distance < min_distance:
                min_distance = distance
                closest_class = class_label

        if min_distance < threshold:
            return closest_class, min_distance
        else:
            return None, min_distance

# Perform anomaly detection using the extracted features
def perform_anomaly_detection(x_train, y_train, x_test, y_test , feature_extractor,):
    prototype_manager = PrototypeManager()
    train_features = extract_features(feature_extractor, x_train)
    test_features = extract_features(feature_extractor, x_test)

# Train the model by prototypes from the training set
    for i, feature_vector in enumerate(train_features):
        class_label = y_train[i]
        prototype_manager.update_or_create_prototype(class_label, feature_vector)

# Test the model and detect anomalies
    predictions = []
    anomalies_detected = 0

    for i, test_feature in enumerate(test_features):
        closest_class, distance = prototype_manager.find_closest_prototype(test_feature)

        if closest_class is not None:
            predictions.append(closest_class)
        else:
            predictions.append(-1)
            anomalies_detected += 1

    accuracy = accuracy_score(y_test, predictions)
    print(f"Anomalies detected: {anomalies_detected}")
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

if __name__ == "__main__":
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    feature_extractor = initialize_feature_extractor()
    perform_anomaly_detection(feature_extractor, x_train, y_train, x_test, y_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 262s 138ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 150ms/step
Anomalies detected: 10000
Test Accuracy: 0.00%
